In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import warnings
import re
from ipyvizzu import Data, Config, Style
from ipyvizzustory import Story, Slide, Step
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [ ]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    """
    A general use decorator to unpack a df[subset] of columns
    into a function which expects the values at those columns
    as arguments
    """
    
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        
        # args[0] is a pandas series equal in length as the 
        # df[subset] to which the apply function is applied 
        series = args[0]

        # series.values holds the number of arguments expected
        # by func and is os length len(df[subset].columns)
        return func(*series.values)

    return _unpack_df_columns

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT * from base_info", con)

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
df.shape

In [ ]:
# df.head(1)

In [ ]:
# df["사원유형"].value_counts()

In [ ]:
columns = ['기준일자','회사','사업본부(지원)','부문','부서','소속','직위','사번','성명','성별','주민번호','생년월일','고용형태','사원유형','직군','직급','그룹입사일자','승급일자','채용경로']

In [ ]:
def 고용형태_변환(df):
    회사 = df["회사"]
    고용형태 = df["고용형태"]
    사원유형 = df["사원유형"]
    
    if 회사 == "HDX" or 회사 == "HCE" or 회사 == "HCM":
        
        if 고용형태 == "정규임원"and 사원유형 == "임원":
            return "임원"
        elif 고용형태 == "정규임원"and 사원유형 == "전문위원":
            return "임원"
        elif 고용형태 == "전문위원"and 사원유형 == "전문위원":
            return "임원"
        elif 고용형태 == "계약임원"and 사원유형 == "임원":
            return "임원"
        
        elif 고용형태 == "상근임원"and 사원유형 == "퇴임임원":
            return "퇴임임원"
        elif 고용형태 == "비상근임원"and 사원유형 == "퇴임임원":
            return "퇴임임원"

        elif 고용형태 == "직원"and 사원유형 == "설계기술직":
            return "직원"
        elif 고용형태 == "직원"and 사원유형 == "연구직":
            return "직원"
        elif 고용형태 == "직원"and 사원유형 == "전문직":
            return "직원"
    
        elif 고용형태 == "일반계약직"and 사원유형 == "별정직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "생산기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "설계기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "연구직":
            return "기간제"
        
        elif 고용형태 == "정년후계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "정년후계약직"and 사원유형 == "생산기술직":
            return "기간제"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "조건제계약직"and 사원유형 == "설계기술직":
            return "기간제"
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "기간제"

        elif 고용형태 == "고용외국인"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "파견후계약직"and 사원유형 == "별정직":
            return "기간제"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "기간제"
        else:
            return 고용형태
    else:
        return 고용형태


In [ ]:
def 사원유형_변환(df):
    회사 = df["회사"]
    고용형태 = df["고용형태"]
    사원유형 = df["사원유형"]
    
    if 회사 == "HDX" or 회사 == "HCE" or 회사 == "HCM":
        
        if 고용형태 == "정규임원"and 사원유형 == "임원":
            return "정규임원"
        elif 고용형태 == "정규임원"and 사원유형 == "전문위원":
            return "전문위원"
        elif 고용형태 == "전문위원"and 사원유형 == "전문위원":
            return "전문위원"
        elif 고용형태 == "계약임원"and 사원유형 == "임원":
            return "계약임원"
        
        elif 고용형태 == "상근임원"and 사원유형 == "퇴임임원":
            return "상근임원"
        elif 고용형태 == "비상근임원"and 사원유형 == "퇴임임원":
            return "비상근임원"

        elif 고용형태 == "직원"and 사원유형 == "설계기술직":
            return "설계연구직"
        elif 고용형태 == "직원"and 사원유형 == "연구직":
            return "설계연구직"
        elif 고용형태 == "직원"and 사원유형 == "전문직":
            return "전문직A"
        elif 고용형태 == "직원"and 사원유형 == "사무지원직":
            return "사무지원/전문직B"
        
        elif 고용형태 == "일반계약직"and 사원유형 == "별정직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "생산기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "설계기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "연구직":
            return "일반계약직"
        
        elif 고용형태 == "정년후계약직"and 사원유형 == "사무기술직":
            return "정년후계약직"
        elif 고용형태 == "정년후계약직"and 사원유형 == "생산기술직":
            return "정년후계약직"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "조건제계약직"and 사원유형 == "설계기술직":
            return "일반계약직"
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "일반계약직"

        elif 고용형태 == "고용외국인"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "파견후계약직"and 사원유형 == "별정직":
            return "파견후계약직"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "일반계약직"
        else:
            return 사원유형
        
    elif 회사 == "HDI":
        if 고용형태 == "직원"and 사원유형 == "전문직B":
            return "사무지원/전문직B"
        elif 고용형태 == "직원"and 사원유형 == "별정직A":
            return "별정직"
        elif 고용형태 == "직원"and 사원유형 == "별정직B":
            return "별정직"
        else:
            return 사원유형
        
    else:
        return 사원유형


In [ ]:
@unpack_df_columns
def get_age(기준일자, birth_year):
    age = int(기준일자[1:5]) - int(birth_year[:4])
    return age

In [ ]:
def add_age_range(age):
    return str(int(np.floor(age/10)*10))+"대"

In [ ]:
def 직급간소화_변환(직급):
    if 직급 == "과장급":
        직급 = "HL3(1)"
    elif 직급 == "상무보급":
        직급 = "상무급"
    elif 직급 == "HL3":
        직급 = "HL3(1)"
    elif 직급 == "HL4":
        직급 = "HL3(2)"
    elif 직급 == "HL5":
        직급 = "HL3(3)"
    
    return 직급

In [ ]:
def 현대인프라팀명추출(df):
    사업본부 = df["사업본부(지원)"]
    회사 = df["회사"]
    소속 = df["소속"]
    부서 = df["부서"]
    pattern1 = r'\b\w+(?:팀)'
    pattern2 = r'\b\w+(?:법인)'
    pattern3 = r'\b\w+(?:부)'

    if 회사 == "HDI":
        # "~부"를 추출하기 위해 "~본부"를 삭제하고 특수문제 &도 삭제한다.
        소속 = 소속.replace("본부", "").replace("부문", "").replace("&","").replace("/","")
        소속 = 소속.replace("이큐브솔루션", "이큐브솔루션팀")
        소속 = 소속.replace("HDEC", "HDEC팀")
        소속 = 소속.replace("HRBP", "HRBP팀")

        
        팀명 = re.findall(pattern1, 소속)   # 팀으로 끝나는 거 추출
        if len(팀명)  == 1:
            return f"{사업본부}_{팀명[0]}"
        else:   # 그런데.. 추출된게 없으면,
            팀명 = re.findall(pattern2, 소속)      # 법인으로 끝나는 거 추출
            
            if len(팀명) == 1:                 
                return f"{사업본부}_{팀명[0]}"     # 법인으로 추출된게 있으면... 리턴해라..
            else:
                팀명 = re.findall(pattern3, 소속)   # 법인으로도 추출된게 없으면.. 부로 끝나는 것 추출
                
                if len(팀명) == 1:           
                    return f"{사업본부}_{팀명[0]}"   # 부으로 끝나는 것 추출된게 있으면 리턴
                else:
                    return 소속
    else:
        return 부서

In [ ]:
def get_승급년도(승급일자):
    try:
        return 승급일자[:4]
    except:
        pass

In [ ]:
@unpack_df_columns
def get_승급년차(기준일자, 승급년도):
    try:
        if 승급년도 is None:
            return 
        
        else:
            return int(기준일자[1:5]) - int(승급년도) + 1
    except:
        pass

## base DF1 생성

In [ ]:
# 퇴임임원 자문역 날리기
df1 = df[columns]
df1.reset_index(drop=True, inplace=True)   # index 새로 부여
# df1["사원유형"].unique()

In [ ]:
df1.info()

## 겸직발령인원 제거하기(HDI, HCE 각각)
- 제뉴인의 재정팀 및 내부회계 겸직자는 DB에서 날림

In [ ]:
# HDI 겸직발령 임직원 인덱스 추출후 드랍
drop_df1 = df1.loc[(df1["회사"] == "HDI") & (df1["사업본부(지원)"] == "현대제뉴인")]
drop_index = drop_df1.index.tolist()
df1 = df1.drop(index=drop_index)

In [ ]:
# 현건ㄴ기 겸직자 전처리 대상 사번
현건기겸직자사번리스트 = ['A524204',
 'A524205',
 'A524206',
 'A524207',
 'A524208',
 'A524209',
 'A524210',
 'A524211',
 'A524212',
 'A524213',
 'A524214',
 'A524215',
 'A524216',
 'A524217',
 'A524218',
 'A524219',
 'A524220',
 'A524221',
 'A524222',
 'A524223',
 'A524224',
 'A524225',
 'A524226',
 'A524227',
 'A524228',
 'A524229',
 'A524230',
 'A524231',
 'A524232',
 'A524233',
 'A524234',
 'A524235',
 'A524236',
 'A524237',
 'A524238',
 'A524239',
 'A524240',
 'A524241',
 'A524242',
 'A524243',
 'A524244',
 'A524245',
 'A524246',
 'A524247',
 'A524248',
 'A524249',
 'A524250',
 'A524251',
 'A524252',
 'A524253',
 'A524254',
 'A524255',
 'A524256',
 'A524257',
 'A524258',
 'A524259',
 'A524260',
 'A524307',
 'A523742',
 'A523767',
 'A523768',
 'A523743',
 'A523744',
 'A523745',
 'A523746',
 'A523748',
 'A523765',
 'A524456',
 'A524457']

In [ ]:
'''
23년 1월 1일 현건기 기준명부상 겸직발령자 식별 불가
23년 2월 8일 발령부터 겸직자 부문명을 "겸직 제뉴인부문"으로 변경
이에 2월 8일 겸직자 사번 기준으로 23년 1월 1일 기준으로 대상자의 부문을 "겸직 제뉴인부문"으로 변경하고
부문명에 "겸직"이 들어간 대상자를 드랍하여 겸직자를 제거함
'''
def 현건기겸직자23년1월부문변경(df):
    기준일자 = df['기준일자']
    사번 = df['사번']
    회사 = df['회사']
    부문 = df['부문']
    global 현건기겸직자사번리스트 

    if 기준일자 == "t20230101" and 회사 == "HCE" and 사번 in 현건기겸직자사번리스트:
        부문 = "겸직 제뉴인부문"
        return 부문
    else:
        return 부문

In [ ]:
df1['부문'] = df1.apply(현건기겸직자23년1월부문변경, axis=1)

In [ ]:
# HCE 부문칼럼에 "겸직"이 있는 행 드랍
df1['부문'].fillna("empty_value", inplace=True)
drop = df1['부문'].str.contains("겸직")
df1 = df1[~drop]

In [ ]:
# 제뉴인은 입사구분에 "계열회사 출입등록" 일괄 삭제
drop_df1 = df1.loc[(df1["회사"] == "HDX")&(df1["채용경로"] == "계열회사 출입등록")]
drop_index = drop_df1.index.tolist()
df1 = df1.drop(index=drop_index)

## 고용형태 및 사원유형 변환

In [ ]:
# df1["사원유형"].value_counts()

In [ ]:
# df1[df1["사원유형"] == "임원"]

In [ ]:
df1["수정_고용형태"] = df1.apply(고용형태_변환, axis=1)
df1["수정_사원유형"] = df1.apply(사원유형_변환, axis=1)
df1["직급"] = df1["직급"].apply(직급간소화_변환)

df1["고용형태"] = df1["수정_고용형태"]
df1["사원유형"] = df1["수정_사원유형"]

In [ ]:
# df1["고용형태"].value_counts()

In [ ]:
df1["사원유형"].value_counts()

In [ ]:
df1.loc[(df1["기준일자"] == "t20230401")&(df1["성명"]=="김범철")]

# 그룹핑 전처리 구간

## 현대인프라팀명 추출후 부서칼럼에 저장 (3사별 조작 칼럼을 동일하게 맞추기 위해)

In [ ]:
# HDI는 부서 칼럼값이 없어서 소속 정보에서 ~"팀" "부"로 끝나는 팀명을 추출하여 부서 칼럼값으로 넣기
df1["부서"] = df1.apply(현대인프라팀명추출, axis=1)  
# df1[df1["회사"]=="HDI"]["부서"].unique()

## 회사별 Unique 팀명 추출후 데이터프레임 통합 (Level1, Level2 그룹핑 검토용)

In [ ]:
# 회사별로 "팀명"과 "회사명" 두개 칼럼으로 구성된 데이터프레임 생성후 Concat

# HDI_team = pd.DataFrame({"부서": df1[df1["회사"]=="HDI"]["부서"].unique().tolist()})
# HDI_team["회사"] = "HDI"

# HCE_team = pd.DataFrame({"부서": df1[df1["회사"]=="HCE"]["부서"].unique().tolist()})
# HCE_team["회사"] = "HCE"

# HDX_team = pd.DataFrame({"부서": df1[df1["회사"]=="HDX"]["부서"].unique().tolist()})
# HDX_team["회사"] = "HDX"

# total_team = pd.concat([HDI_team, HCE_team, HDX_team])
# len(total_team)


In [ ]:
# total_team[total_team["회사명"] == "HDI"]["팀명"].tolist()

In [ ]:
# 검토를 위해 CSV로 저장 후 Level1, Level2 그룹핑 검토 --> 검토후 정리된 파일의 이름을 "team_grouping.csv"로 수정
# total_team.to_csv('total.csv', encoding="utf-8-sig")

## 팀 그룹핑 파일 불러오기

In [ ]:
grouping_df = pd.read_csv("team_grouping.csv", encoding='UTF8')
# grouping_df

In [ ]:
grouping_df["Level1"].unique()

In [ ]:
grouping_df["Level2"].unique()

In [ ]:
# df1.loc[(df1["기준일자"] == "t20230401")&(df1["성명"]=="김종배")]

## DF2에 Left Join

In [ ]:
# 사번대체 임시키 생성
def temp_key(val):
    key = "t"+str(hash(val))
    return key

In [ ]:
# 일부 계열사간 이동 이력 직원의 생일을 양력 기준으로 통일
def new_birthday(kf):
    global df2
    주민번호 = kf["주민번호"]
    생년월일 = kf["생년월일"]
    
    if 주민번호 != None:
        
        if 주민번호[0] == "0":

            temp = "20"+주민번호[:6]
            temp = temp[:4]+"-"+temp[4:6]+"-"+temp[6:8]
            temp = datetime.strptime(temp, '%Y-%m-%d')
        #     temp = temp.strftime('%Y-%m-%d')
            return temp

        else:
            temp = "19"+주민번호[:6]
            temp = temp[:4]+"-"+temp[4:6]+"-"+temp[6:8]
            temp = datetime.strptime(temp, '%Y-%m-%d')
        #     temp = temp.strftime('%Y-%m-%d')
            return temp
    
    else:
        return 생년월일
    

In [ ]:
df2 = pd.merge(df1,grouping_df, on=['회사','부서'], how='left')

In [ ]:
df2.loc[(df2["기준일자"] == "t20230401")&(df2["성명"]=="김범철")]

In [ ]:
# 안쓰는 칼럼 날리고
df2.drop(['부문','열','사업본부(지원)','소속', '직군'], axis=1, inplace=True)
# 팀 상위그룹핑 칼럼 추가
# df2.fillna("", inplace=True)
df2["Level1"].fillna("직할", inplace=True)
df2["Level2"].fillna("직할", inplace=True)
df2["그룹핑"] = df2["Level1"].str.cat(df2["Level2"],sep='_',na_rep="직할")
df2["새생년월일"] = df2.apply(new_birthday, axis=1)
df2["임시키"] = df2["새생년월일"].astype('str')+df2["성명"]
df2["임시키"] = df2["임시키"].apply(temp_key)
df2["연령"] = df2[["기준일자","새생년월일"]].astype('str').apply(get_age, axis=1)
df2["연령대"] = df2["연령"].apply(add_age_range)
df2["승급년도"] = df2["승급일자"].apply(get_승급년도)
df2["승급년차"] = df2[["기준일자","승급년도"]].apply(get_승급년차, axis=1)


In [ ]:
df2.info()

In [ ]:
df2.승급년도.unique()

In [ ]:
df2.승급년차.unique()

In [ ]:
# 중간결과 확인
df2.to_csv("./return_df2.csv",encoding="utf-8-sig")

In [ ]:
df2.loc[(df2["기준일자"] == "t20230401")&(df2["성명"]=="김범철")]

# 중간정리 DF3 (고용형태 : 임원 및 직원)

In [ ]:
df3 = df2.loc[(df2["고용형태"] == "임원") | (df2["고용형태"] == "직원")]
# df3 = df3[df3["사원유형"] != "생산기술직"]
df3.shape

In [ ]:
# df3.head(2)

In [ ]:
# len(df3["사번"].unique()), len(df3["임시키"].unique())

In [ ]:
df3.loc[(df3["기준일자"] == "t20230401")&(df3["성명"]=="김범철")]

In [ ]:
df3.info()

# 중간정리 DF4

In [ ]:
df4 = df3[["임시키", "기준일자","회사", "고용형태", "사원유형", "직급","승급일자","승급년도","승급년차","부서","사번","성명","성별","Level1", "Level2","그룹핑","연령","연령대"]].reset_index(drop=True, inplace=False)
df4["rowid"] = df4.index
# df4.head()

In [ ]:
# df4.info()

In [ ]:
df4.loc[(df4["기준일자"] == "t20230401")&(df4["성명"]=="김범철")]

# 회사별 전입자 / 전출자 리스팅

In [ ]:
직급정렬 = ["부회장급","사장급","부사장급","전무급","상무급","HL3(3)","HL3(2)","HL3(1)","HL2","HL1","S3","S2","S1","사원급" ]
사원유형정렬 = ["정규임원", "전문위원", "계약임원", "상근임원", "비상근임원", "퇴임임원", "설계연구직", "사무기술직", "생산기술직", "전문직A", "전문직","전문직B", "사무지원직", "별정직", "일반계약직", "정년후계약직", "파견후계약직", "고용외국인", "파견직"]

In [ ]:
def 회사별유출인원(회사, before, after):
    global df4
    global 직급정렬
    global 사원유형정렬
    
    before_df = df4.loc[(df4["기준일자"] == before)&(df4["회사"] == 회사)]
    after_df = df4.loc[(df4["기준일자"] == after)&(df4["회사"] == 회사)]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    유출자 = merged_df[merged_df["기준일자_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    
    유출자["사원유형_x"] = pd.Categorical(유출자['사원유형_x'], categories=사원유형정렬, ordered=True)
    유출자["직급_x"] = pd.Categorical(유출자['직급_x'], categories=직급정렬, ordered=True)

    유출자.sort_values(by=["사원유형_x","부서_x","직급_x"], inplace=True)
    return 유출자

In [ ]:
r1 = 회사별유출인원("HDX","t20230101", "t20230401")
r1
len(r1["고용형태_x"])

In [ ]:
def 회사별유입인원(회사, before, after):
    global df4
    global 직급정렬
    global 사원유형정렬
    
    before_df = df4.loc[(df4["기준일자"] == before)&(df4["회사"] == 회사)]
    after_df = df4.loc[(df4["기준일자"] == after)&(df4["회사"] == 회사)]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    유입자 = merged_df[merged_df["기준일자_x"].isnull()][["기준일자_y", "회사_y", "고용형태_y", "사원유형_y", "부서_y","직급_y", "성명_y", "임시키"]]
    
    유입자["사원유형_y"] = pd.Categorical(유입자['사원유형_y'], categories=사원유형정렬, ordered=True)
    유입자["직급_y"] = pd.Categorical(유입자['직급_y'], categories=직급정렬, ordered=True)
    
    유입자.sort_values(by=["사원유형_y", "부서_y", "직급_y"], inplace=True, ascending=True)
    return 유입자

In [ ]:
r2 =  회사별유입인원("HDI","t20230101", "t20230401")
r2["사원유형_y"].value_counts()
len(r2["고용형태_y"])

In [ ]:
r2.head(2)

# 간자(퇴사자 or 건기3사 외부 전적출자) 리스팅

In [ ]:
def 간자리스팅(before, after):
    global df4
    before_df = df4[df4["기준일자"] == before]
    after_df = df4[df4["기준일자"] == after]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    간자 = merged_df[merged_df["고용형태_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    간자 = 간자[간자["고용형태_x"]=="직원"]
    return 간자

In [ ]:
# 간자 = 간자리스팅("t20230101", "t20230401")
# 간자["회사_x"].value_counts()

In [ ]:
# 간자[간자["회사_x"]=="HDX"]

# 온자 리스팅 (입사자, 건기3사 외부에서 전적입자)

In [ ]:
def 온자리스팅(before, after):
    global df4
    before_df = df4[df4["기준일자"] == before]
    after_df = df4[df4["기준일자"] == after]
    merged_df = after_df.merge(before_df, on='임시키',how='outer')
    온자 = merged_df[merged_df["고용형태_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    온자 = 온자[온자["고용형태_x"]=="직원"]
    return 온자

In [ ]:
# 온자 = 온자리스팅("t20230101", "t20230401")
# 온자["회사_x"].value_counts()

In [ ]:
# 온자[온자["회사_x"]=="HDX"]

# 겸직임원 리스팅

In [ ]:
def 겸직임원찾아내기(기준회사, 기준일자):
    global df4
    
    target_df = df4.loc[(df4["기준일자"] == 기준일자)&(df4["회사"] == 기준회사)&(df4["고용형태"]=="임원")]
    compare_df = df4.loc[(df4["기준일자"] == 기준일자)&(df4["회사"] != 기준회사)&(df4["고용형태"]=="임원")]

    겸직임원 = target_df.merge(compare_df, on='임시키',how='inner')  
    겸직임원["직급_x"] = pd.Categorical(겸직임원['직급_x'], categories=직급정렬, ordered=True)
    겸직임원.sort_values(by=["직급_x"], inplace=True)
    
    return 겸직임원[["rowid_x","기준일자_x","임시키", "직급_x", "성명_x", "회사_x", "회사_y"]]

In [ ]:
기준일자들 = df.기준일자.unique().tolist()
회사들 = df.회사.unique().tolist()

In [ ]:
겸직임원들 = []
for 회사 in 회사들:
    for 기준일자 in 기준일자들:
        temp_df = 겸직임원찾아내기(회사, 기준일자)
        겸직임원들.append(temp_df)


In [ ]:
겸직임원_df = pd.concat(겸직임원들)
겸직임원_df.set_index('rowid_x', inplace=True)
# 겸직임원_df[겸직임원_df["성명_x"] == "최은록"]

In [ ]:
겸직임원_df.groupby(["기준일자_x", "회사_x"])["임시키"].count()#.reset_index()

In [ ]:
겸직임원날릴리스트 = 겸직임원_df[겸직임원_df["회사_x"] != "HDX"]
겸직임원날릴리스트.head(2)

In [ ]:
겸직임원날릴리스트 = 겸직임원날릴리스트.index.tolist()
# 겸직임원날릴리스트

In [ ]:
def 겸직임원체크1(id):
    global 겸직임원날릴리스트
    
    if id in 겸직임원날릴리스트:
        return 1
    else:
        return 0

In [ ]:
df4["겸직임원체크"] = df4['rowid'].apply(겸직임원체크1)

# DF5 (최종 결과)

In [ ]:
df5 = df4[['임시키','기준일자','회사','고용형태','사원유형','직급','승급일자','승급년도','승급년차','부서','사번','성명','성별','Level1', 'Level2','그룹핑','연령','연령대','겸직임원체크']]
# df5.head(3)

In [ ]:
df5.to_csv("./return_df5.csv", encoding="utf-8-sig")

In [ ]:
df5.info()

In [ ]:
# 데이터 익명 처리 (사번, 성명 익명화)
df5.drop(["사번", "성명"], axis=1, inplace=True)
df5["성명"] = '박보검'
df5.head(3)

In [ ]:
# df5.loc[(df5["회사"]=="HDX") & (df5["고용형태"] == "임원") & (df5["기준일자"] == "t20230401")]

# 중간결과 저장

In [ ]:
# df5.to_csv("df5.csv", encoding="utf-8-sig")

In [ ]:
import pickle
with open ("pickle_df1.pickle", 'wb') as pickle_filename:
    pickle.dump(df5, pickle_filename)

In [ ]:
pdf1 = pd.pivot_table(df5[df5["회사"]=="HDX"], index=['고용형태','사원유형'], values=['임시키'], columns = ["기준일자"], aggfunc='count')
pdf1

In [ ]:
pdf1.reset_index()